In [38]:
!pip install rasterio

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras import backend as K
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
import threading
import random
import rasterio
import os
import numpy as np
import sys
from sklearn.utils import shuffle as shuffle_lists
from keras.models import *
from keras.layers import *
import numpy as np
from keras import backend as K
from sklearn.model_selection import train_test_split
import joblib

2024-03-19 12:25:55.501967: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 12:25:55.527276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 12:25:55.527315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 12:25:55.527973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 12:25:55.532235: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Unet 모델 정의
def attention_gate(input_x, g, inter_channel):
    # input_x: 스킵 연결로부터의 입력
    # g: 현재 레이어의 입력
    # inter_channel: 내부 채널 수 (필터의 수)

    #input_x => 256, 256, 16

    theta_x = Conv2D(inter_channel, kernel_size=2, strides=2, padding='same')(input_x)
    # theta_x => 128, 128, 16

    # g => 128, 128, 32

    phi_g = Conv2D(inter_channel, kernel_size=1, padding='same')(g)
    # phi_g => 128, 128, 16

    f = Activation('relu')(add([theta_x, phi_g]))

    # f 128 128 16
    psi_f = Conv2D(1, kernel_size=1, padding='same')(f)
    # print(psi_f.shape)

    rate = Activation('sigmoid')(psi_f)

    upsampled_rate = UpSampling2D(size=(2, 2))(rate)
    attended = multiply([input_x, upsampled_rate])
    return attended

def conv2d_block(input_tensor, n_filters, kernel_size=5, batchnorm=True):
    # 기존 conv2d_block 구현을 재사용합니다.
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def get_attention_unet(input_height=256, input_width=256, n_filters=16, dropout=0.1, batchnorm=True, n_channels=10, nClasses=1):
    input_img = Input(shape=(input_height, input_width, n_channels))

    # U-Net의 인코더 부분
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size=5, batchnorm=batchnorm)
    # 256 256 16
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    print(p1.shape)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size=5, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    print(p2.shape)
    
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size=5, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size=5, batchnorm=batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    print(c4.shape)

    c5 = conv2d_block(p4, n_filters * 16, kernel_size=5, batchnorm=batchnorm)
    print(c5.shape)
    # p5 = MaxPooling2D((2, 2))(c5)
    # p5 = Dropout(dropout)(p5)
    # print(c5.shape)
    
    # Attention Gate 적용
    g4 = attention_gate(c4, c5, n_filters * 8)
    up4 = Conv2DTranspose(n_filters*8, (5,5), strides=(2,2), padding='same')(c5)
    up4 = concatenate([up4, g4])
    c6 = conv2d_block(up4, n_filters * 8, kernel_size=5, batchnorm=batchnorm)
    
    g3 = attention_gate(c3, c6, n_filters * 4)
    up3 = Conv2DTranspose(n_filters*4, (5,5), strides=(2,2), padding='same')(c6)
    up3 = concatenate([up3, g3])
    c7 = conv2d_block(up3, n_filters * 4, kernel_size=5, batchnorm=batchnorm)

    g2 = attention_gate(c2, c7, n_filters * 2)
    up2 = Conv2DTranspose(n_filters*2, (5,5), strides=(2,2), padding='same')(c7)
    up2 = concatenate([up2, g2])
    c8 = conv2d_block(up2, n_filters * 2, kernel_size=5, batchnorm=batchnorm)

    g1 = attention_gate(c1, c8, n_filters * 1)
    up1 = Conv2DTranspose(n_filters*1, (5,5), strides=(2,2), padding='same')(c8)
    up1 = concatenate([up1, g1])
    c9 = conv2d_block(up1, n_filters * 1, kernel_size=5, batchnorm=batchnorm)
    
    output_img = Conv2D(nClasses, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[output_img])

    return model

# 모델 생성
model = get_attention_unet(input_height=256, input_width=256, n_filters=16, dropout=0.1, batchnorm=True, n_channels=10, nClasses=1)
model.summary()

2024-03-19 12:29:37.131590: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 881 MB memory:  -> device: 0, name: NVIDIA L40, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-03-19 12:29:37.133900: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43020 MB memory:  -> device: 1, name: NVIDIA L40, pci bus id: 0000:21:00.0, compute capability: 8.9
2024-03-19 12:29:37.135937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43020 MB memory:  -> device: 2, name: NVIDIA L40, pci bus id: 0000:41:00.0, compute capability: 8.9
2024-03-19 12:29:37.137866: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 43020 MB memory:  -> device: 3, name: NVIDIA L40, pci bus id: 0000:61:00.0, compute capability: 8.9
20

(None, 128, 128, 16)
(None, 64, 64, 32)
(None, 32, 32, 128)
(None, 16, 16, 256)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 10)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 16)         4016      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 16)         64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (A

In [5]:
# 두 샘플 간의 유사성 metric
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

# 픽셀 정확도를 계산 metric (Keras Backend 사용)
def pixel_accuracy(y_true, y_pred):
    correct_pixels = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    total_pixels = K.sum(K.round(K.clip(y_true, 0, 1)))
    return correct_pixels / total_pixels

# def pixel_accuracy (y_true, y_pred):
#     sum_n = np.sum(np.logical_and(y_pred, y_true))
#     sum_t = np.sum(y_true)

#     if (sum_t == 0):
#         pixel_accuracy = 0
#     else:
#         pixel_accuracy = sum_n / sum_t
#     return pixel_accuracy



In [6]:

# Adjust the path according to the location of your zip file
zip_file_path = '/content/drive/My Drive/Fire Detection Dataset/train_img.zip'
destination_folder = '/app/'
zip_mask_path = '/content/drive/My Drive/Fire Detection Dataset/train_mask.zip'



In [ ]:
# Unzipping the file
!unzip '{zip_file_path}' -d '{destination_folder}'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_10593.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_9716.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_505.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_24636.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_2643.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_23159.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_22247.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_12384.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_25528.tif  
  inflating: /content/drive/My Drive/Fire Detection Dataset/train_img/train_img_32054.tif  
  inflating: /content/drive/My Drive/Fire Detect

In [13]:
# 사용할 데이터의 meta정보 가져오기

train_meta = pd.read_csv(f'{destination_folder}train_meta.csv')
test_meta = pd.read_csv(f'{destination_folder}test_meta.csv')

#부분적으로 테스트할 때 사용할 수 있는 코드입니다.
# train_meta = train_meta.iloc[0:101]

# 저장 이름
save_name = 'base_line'

N_FILTERS = 16 # 필터수 지정
N_CHANNELS = 3 # channel 지정
EPOCHS = 300 # 훈련 epoch 지정
BATCH_SIZE = 256 # batch size 지정
IMAGE_SIZE = (256, 256) # 이미지 크기 지정
MODEL_NAME = 'attention_unet' # 모델 이름
RANDOM_STATE = 42 # seed 고정
INITIAL_EPOCH = 0 # 초기 epoch

# 데이터 위치
IMAGES_PATH = f'{destination_folder}train_img/'
MASKS_PATH = f'{destination_folder}train_mask/'

# 가중치 저장 위치
OUTPUT_DIR = f'{destination_folder}train_output/'
WORKERS = 4

# 조기종료
EARLY_STOP_PATIENCE = 5

# 중간 가중치 저장 이름
CHECKPOINT_PERIOD = 5
CHECKPOINT_MODEL_NAME = 'checkpoint-{}-{}-epoch_{{epoch:02d}}.hdf5'.format(MODEL_NAME, save_name)

# 최종 가중치 저장 이름
FINAL_WEIGHTS_OUTPUT = 'model_{}_{}_final_weights.h5'.format(MODEL_NAME, save_name)

# 사용할 GPU 이름
CUDA_DEVICE = 3


# 저장 폴더 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


# GPU 설정
os.environ["CUDA_VISIBLE_DEVICES"] = str(CUDA_DEVICE)
try:
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    K.set_session(sess)
except:
    pass

try:
    np.random.bit_generator = np.random._bit_generator
except:
    pass


# train : val = 8 : 2 나누기
x_tr, x_val = train_test_split(train_meta, test_size=0.2, random_state=RANDOM_STATE)
print(len(x_tr), len(x_val))

26860 6715


2024-03-19 12:35:35.326801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 881 MB memory:  -> device: 0, name: NVIDIA L40, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-03-19 12:35:35.328423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43020 MB memory:  -> device: 1, name: NVIDIA L40, pci bus id: 0000:21:00.0, compute capability: 8.9
2024-03-19 12:35:35.330022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43020 MB memory:  -> device: 2, name: NVIDIA L40, pci bus id: 0000:41:00.0, compute capability: 8.9
2024-03-19 12:35:35.331594: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 43020 MB memory:  -> device: 3, name: NVIDIA L40, pci bus id: 0000:61:00.0, compute capability: 8.9
20

In [14]:
from keras import layers

MAX_PIXEL_VALUE = 65535 # 이미지 정규화를 위한 픽셀 최대값

# Define the data augmentation
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.4),
])


class threadsafe_iter:
    """
    데이터 불러올떼, 호출 직렬화
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()


def threadsafe_generator(f):
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))

    return g

def get_img_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    img = np.float32(img)/MAX_PIXEL_VALUE

    return img



def get_img_762bands(path):
    img = rasterio.open(path).read((7,6,2)).transpose((1, 2, 0))
    img = np.float32(img)/MAX_PIXEL_VALUE

    return img

def get_mask_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    seg = np.float32(img)
    return seg





# from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

@threadsafe_generator
def generator_from_lists(images_path, masks_path, batch_size=32, shuffle=True, random_state=None, image_mode='10bands'):
    images = []
    masks = []

    fopen_image = get_img_arr
    fopen_mask = get_mask_arr

    if image_mode == '762':
        fopen_image = get_img_762bands

    i = 0
    while True:
        if shuffle:
            if random_state is None:
                # Assuming shuffle_lists is a custom function that shuffles the paths in unison
                images_path, masks_path = shuffle_lists(images_path, masks_path)
            else:
                images_path, masks_path = shuffle_lists(images_path, masks_path, random_state=random_state + i)
                i += 1

        for img_path, mask_path in zip(images_path, masks_path):

          if not os.path.exists(img_path) or not os.path.exists(mask_path):
            continue

          img = fopen_image(img_path)
          mask = fopen_mask(mask_path)

          # Append original data
          images.append(img)
          masks.append(mask)

          # Apply and append augmented data
          aug_img = data_augmentation(img)  # Apply augmentation to the image
          aug_mask = data_augmentation(mask)  # Apply the same augmentation to the mask
          images.append(aug_img)
          masks.append(aug_mask)

          if len(images) >= batch_size:
              yield (np.stack(images), np.stack(masks))
              images = []
              masks = []

        if len(images) > 0:
            yield (np.stack(images), np.stack(masks))
            images = []
            masks = []



In [15]:
def get_model(model_name, nClasses=1, input_height=256, input_width=256, n_filters = 16, dropout = 0.1, batchnorm = True, n_channels=10):
    # if model_name == 'fcn':
    #     model = FCN
    # elif model_name == 'unet':
    #     model = get_unet
    # elif model_name == 'unet_small':
    #     model = get_unet_small1
    # elif model_name == 'unet_smaller':
    #     model = get_unet_small2
    # elif model_name == 'attention_unet':
    #     model = get_attention_unet
    if model_name == 'attention_unet':
        model = get_attention_unet
    else:
        raise ValueError()

    return model(
            nClasses      = nClasses,
            input_height  = input_height,
            input_width   = input_width,
            n_filters     = n_filters,
            dropout       = dropout,
            batchnorm     = batchnorm,
            n_channels    = n_channels
        )

In [16]:
# train : val 지정 및 generator
images_train = [os.path.join(IMAGES_PATH, image) for image in x_tr['train_img'] ]
masks_train = [os.path.join(MASKS_PATH, mask) for mask in x_tr['train_mask'] ]

images_validation = [os.path.join(IMAGES_PATH, image) for image in x_val['train_img'] ]
masks_validation = [os.path.join(MASKS_PATH, mask) for mask in x_val['train_mask'] ]

train_generator = generator_from_lists(images_train, masks_train, batch_size=BATCH_SIZE, random_state=RANDOM_STATE, image_mode="762")
validation_generator = generator_from_lists(images_validation, masks_validation, batch_size=BATCH_SIZE, random_state=RANDOM_STATE, image_mode="762")


# model 불러오기
model = get_model(MODEL_NAME, input_height=IMAGE_SIZE[0], input_width=IMAGE_SIZE[1], n_filters=N_FILTERS, n_channels=N_CHANNELS)
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='binary_crossentropy', 
              metrics=[dice_coef, pixel_accuracy])
model.summary()


# checkpoint 및 조기종료 설정
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=EARLY_STOP_PATIENCE)
checkpoint = ModelCheckpoint(os.path.join(OUTPUT_DIR, CHECKPOINT_MODEL_NAME), monitor='loss', verbose=1,
save_best_only=True, mode='auto', period=CHECKPOINT_PERIOD)

(None, 128, 128, 16)
(None, 64, 64, 32)
(None, 32, 32, 128)
(None, 16, 16, 256)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_62 (Conv2D)          (None, 256, 256, 16)         1216      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_36 (Ba  (None, 256, 256, 16)         64        ['conv2d_62[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_

In [17]:
print(len(images_train))

26860


In [18]:
print('---model 훈련 시작---')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(images_train) // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=len(images_validation) // BATCH_SIZE,
    callbacks=[checkpoint, es],
    epochs=EPOCHS,
    workers=WORKERS,
    initial_epoch=INITIAL_EPOCH
)
print('---model 훈련 종료---')

---model 훈련 시작---
Epoch 1/300


2024-03-19 12:35:46.265503: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_8/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-19 12:35:56.921400: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.00GiB (rounded to 1073741824)requested by op model_2/conv2d_62/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-19 12:35:56.921560: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-03-19 12:35:56.921591: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 469, Chunks in use: 469. 117.2KiB allocated for chunks. 117.2KiB in use in bin. 50.

ResourceExhaustedError: Graph execution error:

Detected at node model_2/conv2d_62/Conv2D defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 17, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 529, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-pack

압축을 잘못하면... 아래 코드를 사용하세요

In [ ]:
# Import necessary libraries
# from google.colab import drive
import glob
import os

# This will prompt for authorization to access your Google Drive
# drive.mount('/content/drive')

# Define the directory to search in
search_dir = '/content/drive/My Drive/'

# Define the pattern to search for
file_pattern = 'train_img_*.tif'

# Construct the full search path
search_path = os.path.join(search_dir, file_pattern)

# Find all files matching the pattern
matching_files = glob.glob(search_path)

# Optional: Print the files to be deleted (for verification)


# Uncomment the next lines to delete the files
for file in matching_files:
    os.remove(file)
    print(f"Deleted {file}")

print("Deletion complete.")

Deletion complete.
